In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [14]:
TAX_RATE = 1.10

In [15]:
def get_dollar_amt(line):
    index = line.find('$')
    if index != -1:
        line_start = line[index + 1:]
        return float(line_start.replace(",", ""))
    else:
        return None

In [16]:
def get_values(url):
    retail_val = "No retail value found"
    current_val = "Available at retail"

    try:
        result = requests.get(url)
        src = result.content
        soup = BeautifulSoup(src, "html.parser")

        table = soup.find("table")
        if table:
            smalls = table.find_all("small")
            parent_divs = set()

            for small in smalls:
                parent_divs.add(small.find_parent())

            for parent_div in parent_divs:
                text = parent_div.text

                if "$" in text and "Value" in text:
                    current_val = get_dollar_amt(text)
                elif "$" in text:
                    retail_val = get_dollar_amt(text)
    except Exception as e:
        print(f"Error scraping data: {e}")

    return {"retail value": retail_val, "current value": current_val}

In [17]:
def update_prices(data):
    for i, set_num in enumerate(data['Set #']):
        print(f'...updating set #{set_num}')
        url = "https://www.brickeconomy.com/search?query=" + str(set_num)
        set_info = get_values(url)
        data.at[i, 'Current Val.'] = set_info['current value']
        data.at[i, 'Retail Price'] = set_info['retail value']
        if data.at[i, 'Current Val.'] == 'Available at retail':
            data.at[i, 'Highest Val.'] = set_info['retail value']
        else:
            data.at[i, 'Highest Val.'] = max(data.at[i, 'Highest Val.'], data.at[i, 'Current Val.'])

In [22]:
# Purchase Date	Set #	Name	Retail Price	Purchase Price	Qty.	Total Invested	ROI	Current Val.	Highest Val.		Current Invested

# create a dataframe from the csv file
data = pd.read_csv('mycollection.csv')

# update the prices
update_prices(data)

# calculate the current invested
data['Current Invested'] = data['Purchase Price'] * data['Qty.']

data.to_csv('updated_mycollection.csv', index=False)

...updating set #75280
...updating set #75270
...updating set #75274
...updating set #75292
...updating set #75269
...updating set #75283
...updating set #75294
...updating set #75280
...updating set #75294
...updating set #77901
